In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline
import matplotlib


### show laserscan   'new' type (hdf5 datafile; fast_laser_scan_green_red.py)

In [ ]:
######show and fit laserscan, from fast_laser_scan_green_red.py

nr_of_scans =1
older_than=None#'20161130163600'#None#
#fig,ax=plt.subplots(1, figsize=(16,5))
for i in range(nr_of_scans):
    timestamp,folder = tb.latest_data(contains='112107_LaserScanGreenRed_Lily_NV5_g_30.0_r_8.0', older_than = older_than,return_timestamp=True)
    older_than=timestamp

    a=m2.M2Analysis(folder=folder)
    for k in a.f.keys():
        if '_data' in k:
            a.dg = a.f[k]
    a.counts,a.frequency,a.voltage = a.dg['Counts [Hz]'].value,a.dg['Frequency [GHz]'].value,a.dg['Voltage [V]'].value
    #print len(a.counts)
    #if len(a.counts)>200:
        
    f_min = 92
    f_max = 180

    idx_min = np.searchsorted(a.frequency, f_min, side="left")
    idx_max = np.searchsorted(a.frequency, f_max, side="left")
    frequency = a.frequency[idx_min:idx_max]
    counts = a.counts[idx_min:idx_max]

    fig,ax=plt.subplots(1, figsize=(10,5))
    ax.plot(frequency, counts, label=' '+a.name)

    ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
    ax.set_ylabel("Countrate (Hz)", fontsize = 14)
    ax.set_title(os.path.split(folder)[-1])
    plt.savefig(os.path.join(folder,'plot_laserscan.png'))
    plt.show()

    a.finish()
        
# ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
# ax.set_ylabel('Countrate [Hz]')
# #ax.legend()




### fit (lorentzian to) laserscan 'new' type (hdf5 datafile; fast_laser_scan_green_red.py)

In [ ]:
######show and fit laserscan, from fast_laser_scan_green_red.py

nr_of_scans =1
older_than='20161130163600'#None#
#fig,ax=plt.subplots(1, figsize=(16,5))
for i in range(nr_of_scans):
    timestamp,folder = tb.latest_data(contains='105435_LaserScanGreenRed_Lily_NV5_g_30.0_r_8.0', older_than = older_than,return_timestamp=True)
    older_than=timestamp

    #print folder

    a=m2.M2Analysis(folder=folder)
    for k in a.f.keys():
        if '_data' in k:
            a.dg = a.f[k]
    a.counts,a.frequency,a.voltage = a.dg['Counts [Hz]'].value,a.dg['Frequency [GHz]'].value,a.dg['Voltage [V]'].value
    #print len(a.counts)
    #if len(a.counts)>200:
    #    ax.plot(a.frequency, a.counts, label=' '+a.name)
        
    f_min = 92
    f_max = 180
    g_a= 200
    g_A= 200
    g_x0 = 72
    g_gamma = 0.5
    #g_A2= 200
    #g_x02 = 84
    #g_gamma2 = 0.5
    fixed=[]

    idx_min = np.searchsorted(a.frequency, f_min, side="left")
    idx_max = np.searchsorted(a.frequency, f_max, side="left")
    frequency = a.frequency[idx_min:idx_max]
    counts = a.counts[idx_min:idx_max]

    fig,ax=plt.subplots(1, figsize=(10,5))

    p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
    #p0, fitfunc, fitfunc_str = common.fit_2lorentz(g_a, g_A, g_x0, g_gamma,g_A2, g_x02, g_gamma2)
    fit_result = fit.fit1d(frequency,counts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

    T = fit_result['params_dict']['x0']
    gamma = fit_result['params_dict']['gamma']
    u_gamma = fit_result['error_dict']['gamma']

    print 'frequency = ', T , 'GHz'
    print 'linewidth= ', gamma,'+/-', u_gamma, 'GHz'

    plot.plot_fit1d(fit_result, np.linspace(min(frequency),max(frequency),len(frequency)),ax=ax, label='Fit',show_guess=False, plot_data=True,color='red', data_linestyle = '-', print_info= False)
    ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
    ax.set_ylabel("Countrate (Hz)", fontsize = 14)
    ax.set_title(os.path.split(folder)[-1])
    ax.legend(title='Transition at %.2f GHz, Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3))
    plt.savefig(os.path.join(folder,'fit_laserscan.png'))
    plt.show()

    a.finish()
        
# ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
# ax.set_ylabel('Countrate [Hz]')
# #ax.legend()





### fitting for 'old' laserscan file (.dat data type; fast_laser_scan.py)

In [ ]:
### Fitting laserscans 1 peak at a time
font = {'size'   : 22}
matplotlib.rc('font', **font)

f_min = 57.0
f_max = 59.0
g_a= 400
g_A= 20
g_x0 = 57.9
g_gamma = 0.1
fixed=[]


folder = r'D:\measuring\data\20170215\153242_laser_scanSirius_day2spot1_g_10.0_r_8.0_0'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
if len(d)>2:
    voltage, frequency, counts = d[:,0],d[:,1],d[:,2]

fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(frequency,counts,'-',color='red')
ax.set_title(folder)
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
plt.savefig(os.path.join(folder,'Scan5_NV1_wholefigure.png'))

if f_min ==None:
    f_min = min(frequency)
if f_max == None: 
    f_max = max(frequency)
    
idx_min = np.searchsorted(frequency, f_min, side="left")
idx_max = np.searchsorted(frequency, f_max, side="left")
frequency = frequency[idx_min:idx_max]
counts = counts[idx_min:idx_max]

fig,ax=plt.subplots(1, figsize=(10,5))
    
p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(frequency,counts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

T = fit_result['params_dict']['x0']
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']

print 'T = ', T , 'GHz'
print 'Linewidth T', gamma, 'GHz +/-', u_gamma, 'GHz'

plot.plot_fit1d(fit_result, np.linspace(min(frequency),max(frequency),len(frequency)),ax=ax, label='Fit',show_guess=False, plot_data=True,color='red', data_linestyle = '-', print_info= False)
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)")#, fontsize = 14)
ax.set_ylabel("Countrate (Hz)")#, fontsize = 14)
#ax.set_title(folder)


ax.set_xlim(f_min,f_max)
ax.set_title('Transition at %.2f GHz,\n Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3))
#ax.legend(title='Transition at %.2f GHz, Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3),loc=0)
plt.savefig(os.path.join(folder,'Scan1_NV1_Ey.png'))

# plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
#            ncol=2, mode="expand", borderaxespad=0.)


In [ ]:
nvlevels.get_transitions_ExEy(56,84.62,
                              show_ms0_transitions=True,
                              show_m1_transitions=True,
                              show_p1_transitions=False,
                              show_A_transitions=False,
                              show_FB_A_transitions=False,
                              show_FB_E_transitions=False,
                              show_E_prime_flip_transitions=False)

In [ ]:
########2d plot of repeated laserscans

older_than = '20170215140000'
fig,ax=plt.subplots(1, figsize=(16,5))
fig2,ax2=plt.subplots(1, figsize=(16,5))
bins = np.arange(70,73,.05)
hist = np.zeros(len(bins)-1)
bin_norm = np.zeros(len(bins)-1)
i=0
nr_files = 2


for i in np.arange(nr_files):
    old_laser_scan=False
    ret = tb.latest_data(contains='laser_scanSirius_day2spot1_g_0.0_r_8.0_', older_than=older_than, return_timestamp=True, raise_exc=False)
    if not ret:
        break
    print ret

    timestamp,folder = ret
    older_than=timestamp

    d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
    if len(d)>2:
        voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
    
    if len(counts)>200:
        cts_norm_zero = counts.astype(np.float)/np.average(counts) -1
        _h,_b = np.histogram(frequency, weights=cts_norm_zero, bins=bins)
        bin_norm+=1.0*(_h>0)
        hist+= _h
        ax2.plot(frequency,cts_norm_zero+1, alpha=0.5)
    i+=1
    

ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax2.set_ylabel('Normalized countrates')
#ax2.set_xlim(70,90)
#ax.legend()

In [ ]:
older_than='20170215140000'
newer_than = '20170215131500'
newer_than_old = '20170215131500'
fig,ax=plt.subplots(1, figsize=(16,5))
fig2,ax2=plt.subplots(1, figsize=(16,5))
bins = np.arange(70,73,.05)
hist = np.zeros(len(bins)-1)
bin_norm = np.zeros(len(bins)-1)
i=0
nr_files = 30

for i in np.arange(nr_files):
    print i
    old_laser_scan=False
    ret = tb.latest_data(contains='LaserScan', older_than=older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    if not ret:
        ret = tb.latest_data(contains='laser_scanSirius_day2spot1_g_0.0_r_8.0_'+str(i+1),older_than=older_than,newer_than=newer_than_old, return_timestamp=True, raise_exc=False)
        old_laser_scan = True
        if not ret:
            break
    print d

    timestamp,folder = ret
    older_than=timestamp
    if old_laser_scan:
        d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
        if len(d)>2:
            voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
    else:
        a=m2.M2Analysis(folder=folder)
        data_found = False
        for k in a.f.keys():
            if '_data' in k and 'area_5' in k:
                a.dg = a.f[k]
                data_found=True
        if not data_found:
            continue
        voltage, frequency, counts = a.dg['Voltage [V]'].value, a.dg['Frequency [GHz]'].value, a.dg['Counts [Hz]'].value
     
    if len(counts)>200:
        cts_norm_zero = counts.astype(np.float)/np.average(counts) -1
        _h,_b = np.histogram(frequency, weights=cts_norm_zero, bins=bins)
        bin_norm+=1.0*(_h>0)
        hist+= _h
        ax2.plot(frequency,cts_norm_zero+1, alpha=0.5)
    i+=1
    
ax.plot(bins[1:],hist, '.')        
#ax.set_xlim(70,90)
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Deviation from average countrates')
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax2.set_ylabel('Normalized countrates')
#ax2.set_xlim(70,90)
#ax.legend()

In [ ]:
older_than='20170215140000'
newer_than = '20170215131500'
fig,ax=plt.subplots(1, figsize=(16,5))
fig2,ax2=plt.subplots(1, figsize=(16,5))
fig4,ax4=plt.subplots(1,figsize=(15,15))

f_min = 70.6
f_max = 71.5
bins = np.arange(f_min,f_max,.01)
hist = np.zeros(len(bins)-1)
hist_2D=np.empty((0, len(bins)-1))
bin_norm = np.zeros(len(bins)-1)
n_xticks =10 # every n_ticks bins there will be a tick on the axis
ii = 0


while 1:
    ret = tb.latest_data(contains='laser_scan', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    if not ret:
        break
    timestamp,folder = ret
    older_than=timestamp
    if not 'g_0.0' in folder:
        print folder
        continue        
    #print folder
    d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
    if len(d)>2:
        voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
    if len(counts)>100:
        cts = counts.astype(np.float)/np.average(counts) -1
        _h,_b = np.histogram(frequency, weights=counts, bins=bins)
        hist+= _h
        hist_2D = np.append(hist_2D, [_h], axis=0) 
        ax2.plot(frequency,cts, alpha=0.5)
        ii+=1
        
ax.plot(bins[1:],hist/ii)        
#ax.set_xlim(min(bins),max(bins))
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Average counts [Hz]')

g_a= 400
g_A= 20
g_x0 = 70.9
g_gamma = 0.1

p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(bins[1:],hist/ii, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)
plot.plot_fit1d(fit_result, np.linspace(min(frequency),max(frequency),len(frequency)),ax=ax, label='Fit',
                show_guess=False, plot_data=True,color='red', data_linestyle = '-', print_info= False)

T = fit_result['params_dict']['x0']
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']

ax.set_xlim(f_min,f_max)
ax.set_title('Transition at %.2f GHz,\n Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3))

ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax2.set_ylabel('Normalized countrates')
ax2.set_xlim(min(bins),max(bins))
#ax.legend()
print ii

# Spectral diffusion plot
#ax4.set_title(timestamp+'_'+os.path.split(folder)[-1])



ax4.set_xlabel('Frequency (GHz wrt 470.4 THz)', fontsize = 18)
ax4.set_ylabel('Scan number', fontsize = 18)

ax4.set_ylim(0,ii)
#ax4.set_xlim(min(bins),max(bins))
#xticklabels = np.linspace(min(bins),max(bins),n_xticks)
ax4.set_xticklabels(bins[0::n_xticks])
ax4.pcolormesh(hist_2D, cmap='Greys_r', alpha = 1)


# # getting the linewidths of both datasets and deduce spectral diffusion
# print 'Linewidth of normal data = ', round(gamma*1.e3),'+/-',round(u_gamma*1.e3), 'MHz'
# print 'Linewidth of shifted data = ', round(gamma1*1.e3),'+/-',round(u_gamma1*1.e3), 'MHz'
# if abs(gamma - gamma1) > u_gamma1:
#         print 'Estimate of spectral diffusion = ', round(gamma*1.e3 - gamma1*1.e3), 'MHz'
# else:
#     print 'errorbars larger than difference between linewidth, no estimate of spectral diffusion possible. '
# plt.show()

In [ ]:
# calibrate laser w.r.t. spectrometer 
t = np.array([637.009, 637.04,637.06,637.153,637.289])
d = np.array([250, 232.1, 217.1, 153.4,49.4])

x,y=1/t,d
fig,ax=plt.subplots(1, figsize=(16,5))

f = common.fit_line
args=[0,100]
fitres = fit.fit1d(x, y, f,*args, fixed = [],
                   do_print = True, ret = True, maxfev=100)
p1 = fitres['params_dict']
ax.plot(x,y, 'o')
plot_pts=100
x_p=np.linspace(min(x),max(x),plot_pts)
if fitres['success']:
    f_p = fitres['fitfunc'](x_p)
    ax.plot(x_p,f_p)
f_g = f(*args)[1](x_p)
ax.plot(x_p,f_g)

In [ ]:
import mpmath

In [ ]:
folder = r'd:\\measuring\\data\\20160921\\161331_laser_scan_Sophie_area_5_NV1_g_2.0_r_200.0'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
if len(d)>2:
    frequency, counts,counts_no_green = d[:,0],d[:,1],d[:,2]

In [ ]:
fig,(ax,ax2)=plt.subplots(2, figsize=(16,8))
ax.plot(frequency,counts, label = 'Green + Red')
ax.plot(frequency, counts_no_green, label = 'Red only')
ax2.plot(frequency,counts-counts_no_green,'r',
         label = 'Difference')
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Countrates [Hz]')
ax2.set_ylabel('Countrate difference [Hz]')
ax.legend()
ax2.legend()
#from analysis.lib.tools import rebin
#order = 1
#ax.plot(rebin.average(frequency,order=order),rebin.rebin(counts-counts_no_green,order=order))

In [ ]:
bins = np.arange(200,1050,.5)

hist,_b = np.histogram(frequency, weights=counts-counts_no_green, bins=bins)       
fig,ax2=plt.subplots(1, figsize=(16,5))
ax2.plot(bins[1:],hist, '.')   

In [ ]:
print 91.18 - 79.23